In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import time

plt.rcParams['font.family'] = 'AppleGothic'
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# ✅ ORB 특징점 수 줄이기 (nfeatures=100)
orb = cv2.ORB_create(nfeatures=100)

def detect_face(image):
    """ 얼굴 검출 후 150x150 크기로 리사이즈 """
    try:
        if image is None:
            return None
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
        if len(faces) > 0:
            x, y, w, h = faces[0]
            return cv2.resize(image[y:y+h, x:x+w], (150, 150))
        return None
    except:
        return None

def get_face_embedding(image):
    """ 얼굴 이미지에서 ORB 특징 벡터 추출 """
    try:
        if image is None:
            return None
        keypoints, descriptors = orb.detectAndCompute(image, None)
        return descriptors
    except:
        return None

def process_single_image(img_path):
    """ 단일 이미지 처리 (얼굴 검출 → 특징 벡터 추출) """
    try:
        filename = os.path.basename(img_path)
        img = cv2.imread(img_path)
        if img is None:
            return None
        face = detect_face(img)
        if face is not None:
            embedding = get_face_embedding(face)
            if embedding is not None:
                return filename, embedding, cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        return None
    except:
        return None

def load_or_create_embeddings(data_dir, cache_file="face_embeddings_cache.pkl", batch_size=50):
    """ 이미지 폴더에서 얼굴 임베딩을 추출하고 저장 (Batch 방식) """
    if os.path.exists(cache_file):
        os.remove(cache_file)

    image_files = [f for f in os.listdir(data_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    face_embeddings = {}
    image_data = {}

    print(f"🔄 총 {len(image_files)}개의 이미지 처리 시작...")

    start_time = time.time()

    # ✅ ThreadPoolExecutor 제거 → 순차 처리로 안정성 확보
    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i+batch_size]
        print(f"🛠 Batch {i//batch_size + 1}: {len(batch_files)}개 처리 중...")

        results = []
        for file in batch_files:
            result = process_single_image(os.path.join(data_dir, file))
            if result:
                results.append(result)

        for filename, embedding, face_img in results:
            face_embeddings[filename] = embedding
            image_data[filename] = face_img

    print(f"✅ 모든 이미지 처리 완료: {len(face_embeddings)}개 얼굴 탐지됨, {time.time() - start_time:.2f}초 소요")

    with open(cache_file, 'wb') as f:
        pickle.dump({'embeddings': face_embeddings, 'images': image_data}, f)

    return face_embeddings, image_data

def find_most_similar_face_batch(target_embedding, face_embeddings, top_n=5, batch_size=50):
    """ 타겟 얼굴과 가장 유사한 얼굴 찾기 (Batch 방식) """
    if target_embedding is None or len(face_embeddings) == 0:
        return []

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    results = []

    embedding_items = list(face_embeddings.items())

    for i in range(0, len(embedding_items), batch_size):
        batch = embedding_items[i:i+batch_size]

        for name, embedding in batch:
            try:
                if embedding is not None:
                    matches = bf.match(np.uint8(target_embedding), np.uint8(embedding))
                    avg_distance = sum(m.distance for m in matches) / len(matches) if matches else float('inf')
                    similarity = max(0, 100 - min(100, avg_distance))
                    results.append((name, similarity))
            except:
                continue

    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_n]

def main():
    try:
        data_dir = "images1/"
        target_path = "target2.jpg"

        target_img = cv2.imread(target_path)
        if target_img is None:
            print("❌ 타겟 이미지를 불러올 수 없습니다.")
            return

        target_face = detect_face(target_img)
        if target_face is None:
            print("❌ 타겟 이미지에서 얼굴을 찾을 수 없습니다.")
            return

        target_embedding = get_face_embedding(target_face)
        if target_embedding is None:
            print("❌ 타겟 얼굴에서 특징점을 추출할 수 없습니다.")
            return

        # ✅ 안정성을 위한 순차적 얼굴 데이터셋 처리
        face_embeddings, image_data = load_or_create_embeddings(data_dir, batch_size=50)

        # ✅ 안정성을 위한 순차적 유사 얼굴 찾기
        top_matches = find_most_similar_face_batch(target_embedding, face_embeddings, top_n=5, batch_size=50)

        if not top_matches:
            print("❌ 유사한 얼굴을 찾을 수 없습니다.")
            return

        # 최종 결과 출력
        print("\n🔍 가장 닮은 얼굴 리스트:")
        for i, (match_name, similarity) in enumerate(top_matches, 1):
            print(f"{i}. {match_name} - 유사도 {similarity:.2f}%")

        # 시각화
        fig, axes = plt.subplots(1, min(6, len(top_matches) + 1), figsize=(15, 5))
        axes[0].imshow(cv2.cvtColor(target_face, cv2.COLOR_BGR2RGB))
        axes[0].axis("off")
        axes[0].set_title("🎯 타겟 이미지")

        for i, (match_name, similarity) in enumerate(top_matches):
            if match_name in image_data:
                axes[i+1].imshow(image_data[match_name])
                axes[i+1].axis("off")
                axes[i+1].set_title(f"{similarity:.2f}%")

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"❌ 프로그램 실행 중 오류: {e}")

if __name__ == "__main__":
    main()
